# 02 - Model Training and Walk-Forward

This notebook shows how to:

- Build training tensors from OHLCV data
- Train LSTM / TCN / Transformer models
- Run a simple walk-forward backtest
- Inspect basic performance metrics



In [ ]:
import pathlib

import numpy as np

from research.common import load_ohlcv_csv, BarConfig
from research.feature_engineering import FeatureConfig, build_features
from research.model_lstm import LSTMAlpha, LSTMConfig, train_lstm
from research.model_tcn import TCNAlpha, TCNConfig, train_tcn
from research.model_transformer import TransformerAlpha, TransformerConfig, train_transformer
from research.walk_forward import WalkForwardConfig, walk_forward
from research.evaluation import aggregate_walk_forward

DATA_PATH = pathlib.Path("../data/equities/your_asset.csv")

raw = load_ohlcv_csv(str(DATA_PATH))
bar_cfg = BarConfig(lookback_window=128, prediction_horizon=5)
feat_cfg = FeatureConfig(bar=bar_cfg)

X, y, feature_cols = build_features(raw, feat_cfg)
X.shape, y.shape


In [ ]:
# Choose a model family here: "lstm", "tcn", or "transformer"
MODEL_FAMILY = "lstm"

n_features = X.shape[-1]


def train_fn(train_data, val_data):
    X_train, y_train = train_data
    X_val, y_val = val_data

    if MODEL_FAMILY == "lstm":
        cfg = LSTMConfig(input_size=n_features)
        model = LSTMAlpha(cfg)
        return train_lstm(model, (X_train, y_train), (X_val, y_val), epochs=5)
    elif MODEL_FAMILY == "tcn":
        cfg = TCNConfig(input_size=n_features)
        model = TCNAlpha(cfg)
        return train_tcn(model, (X_train, y_train), (X_val, y_val), epochs=5)
    elif MODEL_FAMILY == "transformer":
        cfg = TransformerConfig(input_size=n_features)
        model = TransformerAlpha(cfg)
        return train_transformer(model, (X_train, y_train), (X_val, y_val), epochs=5)
    else:
        raise ValueError(f"Unknown MODEL_FAMILY={MODEL_FAMILY}")



In [ ]:
wf_cfg = WalkForwardConfig(window_train=2000, window_val=500, step=250)

wf_results = walk_forward(X, y, wf_cfg, train_fn)
len(wf_results)


In [ ]:
summary = aggregate_walk_forward(wf_results)
stats = summary["stats"]

stats
